# LangChain Context-Aware AI: Introduction to RAG

## Overview
This notebook introduces context-aware AI and the fundamentals of RAG (Retrieval-Augmented Generation). You'll learn how to provide specific context to AI models to get more accurate and reliable responses.

## Learning Objectives
- Understand context-aware prompting and its benefits
- Learn RAG (Retrieval-Augmented Generation) principles
- Build prompts that use external context effectively
- Handle cases where context doesn't contain requested information
- Prevent AI hallucination with context-based constraints

## Key Concepts
- **Context-Aware AI**: Providing specific information to guide AI responses
- **RAG**: Retrieval-Augmented Generation for grounded AI answers
- **Context Constraints**: Limiting AI responses to provided information only
- **Fallback Responses**: Handling insufficient context gracefully

## Step 1: Environment Setup

This notebook provides the same two configuration options as previous notebooks:
- **Azure OpenAI** (commented out) for cloud-based AI
- **LM Studio** (active) for local AI inference

We'll continue using LM Studio for local development and learning.

In [ ]:
# # with Azure OpenAI and LangChain

# from langchain_openai import AzureChatOpenAI           # Azure OpenAI integration
# from langchain_core.prompts import ChatPromptTemplate  # For structured prompts
# from langchain_core.output_parsers import StrOutputParser  # For cleaning outputs

# # Message types for conversation structure
# from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# # Environment and configuration
# from dotenv import load_dotenv
# import os

# # Load environment variables from .env file
# load_dotenv()

# # Initialize Azure OpenAI client with credentials
# llm = AzureChatOpenAI(
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),          
#     azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"), 
#     api_version=os.getenv("AZURE_OPENAI_API_VERSION"),          
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"),                  
# )

### Option 1: Azure OpenAI Configuration (Commented Out)
Cloud-based AI configuration for production environments.

### Option 2: LM Studio Configuration (Active)
Local AI configuration for development and learning. Ensure LM Studio is running with a model loaded. 

In [1]:
# with local LM Studio and LangChain

from langchain_openai import ChatOpenAI           # Azure OpenAI integration
from langchain_core.prompts import ChatPromptTemplate  # For structured prompts
from langchain_core.output_parsers import StrOutputParser  # For cleaning outputs

# Message types for conversation structure
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Environment and configuration
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Initialize LM Studio client
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="not-needed"  # LM Studio doesn't require an API key
)

## Step 2: Building a RAG-Ready Prompt Template

This prompt template is designed for RAG (Retrieval-Augmented Generation) workflows. It includes:

### Key RAG Features:
- **Context Integration**: Dedicated space for external information
- **Strict Constraints**: AI must use only provided context
- **Fallback Handling**: Clear response when context is insufficient
- **Hallucination Prevention**: Prevents AI from making up information

### Template Structure:
- **System Instructions**: Define AI behavior and constraints
- **Context Section**: Where retrieved information goes
- **Question Section**: User's actual query
- **Reminder**: Reinforces context-only responses

In [2]:
prompt_template = ChatPromptTemplate.from_template(
    """you are a helpful and comic assistant that help users to find the answers in fun way based on given context.
       
       IMPORTANT RAG INSTRUCTIONS:
       - Use ONLY the information provided in the context below
       - If the context doesn't contain enough information to answer the question, say: "I don't have enough info on this :("
       - Do not make up information or use knowledge outside the provided context
       - Be helpful and engaging while staying factually accurate
       
       ----
       Context: {context}
       ----
       Question: {question}
       
       Remember: Base your answer ONLY on the context provided above!
    """
)

## Step 3: Creating the Processing Chain

Building our familiar three-component chain for clean, processed output:
`prompt_template | llm | parser`

In [3]:
parser = StrOutputParser()
chain = prompt_template | llm | parser

## Step 4: Setting Up Context Data

In real RAG systems, this context would come from:
- **Vector databases** (similarity search)
- **Document retrieval** (relevant text chunks)
- **Knowledge bases** (structured information)

For learning purposes, we'll use a simple string with basic moon facts.

In [8]:
context_data = "moon is earth's natural satelite and orbit around it in 28 days cycle"

## Step 5: Testing Context-Aware Responses

Let's test our RAG system with three different scenarios:

### Test 1: Information Available in Context
Asking about the moon - our context contains this information.

In [9]:
question = "what is moon"
response = chain.invoke({"context": context_data, "question": question})
print(response)  # Print the content of the response

Okay, buckle up for some lunar knowledge! 🌕

According to my super-secret intel (aka the context you gave me!), the moon is Earth's natural satellite! It zooms around our planet in a cycle that takes about 28 days. ✨

Pretty neat, huh? Though I gotta say, my info is a *little* limited... I don't have all the details on what it's made of or if there are little green cheese people living there! 🧀👽






### Test 2: Partial Information in Context
Asking about Earth - our context mentions Earth but doesn't fully describe it.

In [6]:
question = "what is earth"
response = chain.invoke({"context": context_data, "question": question})
print(response)  # Print the content of the response

Okay, buckle up for some Earth-shattering (not literally, we're keeping it chill!) info! 

According to my super-secret documents (aka the context you gave me!), Earth has a natural satellite called... the Moon! And that moon orbits around it. 

So, Earth is a planet that has a moon orbiting around it in a 28-day cycle. Pretty neat, huh? ✨🌙






### Test 3: Information NOT in Context
Asking about the sun - our context contains no information about the sun. This tests our fallback response.

In [10]:
question = "where is sun?"
response = chain.invoke({"context": context_data, "question": question})
print(response)  # Print the content of the response

Oh, that's a tricky one! Based on what I know (which is just this little bit of info!), the context doesn't say anything about where the Sun is. It only talks about the Moon and Earth! 

I don't have enough info on this :(. Maybe ask me something about our lunar friend? I'm a Moon expert, apparently! 😉



## Step 6: Analyzing RAG Behavior

### What We Observed:

**✅ Context Available**: AI provides accurate answer based on provided information  
**⚠️ Partial Context**: AI works with limited information or indicates insufficient data  
**❌ No Context**: AI properly responds with "I don't have enough info" instead of hallucinating  

### Why This Matters:

- **Accuracy**: Responses are grounded in actual data
- **Reliability**: No false information from AI hallucination
- **Transparency**: Clear when information is insufficient
- **Trust**: Users know responses are fact-based

## Summary

You've successfully learned the fundamentals of context-aware AI and RAG! You've mastered:

✅ **RAG Prompt Design**: Created prompts that integrate external context  
✅ **Context Constraints**: Limited AI responses to provided information only  
✅ **Fallback Handling**: Managed cases with insufficient context gracefully  
✅ **Hallucination Prevention**: Stopped AI from making up information  
✅ **Reliable AI**: Built trustworthy AI responses grounded in facts  

## Key Concepts Learned

- **RAG (Retrieval-Augmented Generation)**: Enhancing AI with external context
- **Context Integration**: Providing specific information to guide AI responses
- **Strict Constraints**: Using "ONLY" instructions to prevent hallucination
- **Fallback Responses**: Graceful handling when context is insufficient
- **Grounded AI**: AI responses based on factual information

## RAG Benefits

- **Accuracy**: Responses based on actual data, not general training
- **Up-to-date**: Can include recent information not in training data
- **Domain-specific**: Tailored responses for specific knowledge areas
- **Trustworthy**: Clear source of information for responses
- **Controllable**: Predictable behavior within context boundaries

## Real-World Applications

- **Customer Support**: Answer questions using company documentation
- **Research Assistant**: Provide answers from specific research papers
- **Legal AI**: Respond based on specific case law and regulations
- **Medical AI**: Answer questions using current medical literature
- **Education**: Teach concepts using specific curriculum materials

## Next Steps

In the next notebook, you'll explore:
- **Embeddings**: Converting text to numerical vectors for similarity search
- **Vector Similarity**: Finding relevant context automatically
- **Advanced RAG**: Building smarter context retrieval systems